In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0506_sec 2a.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(99315, 29)


,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
83594,3.232030e+09,3,2,32320302,54,NaN,NaN,NaN,1,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
45916,2.051110e+09,2,1,20511102,1,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
79887,3.162040e+09,3,2,31620401,52,2.0,NaN,2.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
55318,2.072030e+09,2,2,20720301,4,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
29847,1.222030e+09,1,2,12220301,5,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [2]:
# example of ONE HOUSEHOLD (idc separates individuals within households)
C.loc[C['hhcode']==1011010105]

,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
16,1.011010e+09,1,1,10110101,1,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
17,1.011010e+09,1,1,10110101,2,1.0,1.0,1.0,3,NaN,...,1.0,5.0,5.0,1.0,4.0,5.0,1.0,1800.0,2000.0,3800.0
18,1.011010e+09,1,1,10110101,3,1.0,1.0,1.0,3,NaN,...,1.0,5.0,4.0,1.0,3.0,5.0,1.0,1800.0,2000.0,3800.0
19,1.011010e+09,1,1,10110101,51,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
20,1.011010e+09,1,1,10110101,52,1.0,1.0,1.0,3,NaN,...,1.0,5.0,8.0,1.0,7.0,5.0,1.0,0.0,0.0,1250.0
21,1.011010e+09,1,1,10110101,53,1.0,1.0,1.0,3,NaN,...,1.0,5.0,6.0,1.0,5.0,5.0,1.0,0.0,0.0,800.0


In [3]:
# Create a yes/no column for "ever admitted"
C['ever_admitted']= 'yes'
C.loc[(C["s2bq01"] == 1), 'ever_admitted'] = 'no'
C['currently_enrolled']= 'no'
C.loc[(C["s2bq01"] == 3), 'currently_enrolled'] = 'yes'

In [4]:
# Reduce to the questions of interest (4 questions):

C["max_level_acheived"] = C["s2bq05"]
C["why_not"] = C["s2bq02"]
#C['enrolled_last_year']=C["s2bq05"]

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [5]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.011010e+09,1,no,NaN,no,NaN,1,1
1,1.011010e+09,51,yes,17.0,no,NaN,1,1
2,1.011010e+09,1,yes,12.0,no,NaN,1,1
3,1.011010e+09,51,yes,16.0,no,NaN,1,1
4,1.011010e+09,52,yes,NaN,yes,NaN,1,1
...,...,...,...,...,...,...,...,...
99310,4.071340e+09,52,yes,NaN,yes,NaN,1,4
99311,4.071340e+09,1,yes,8.0,no,NaN,1,4
99312,4.071340e+09,51,no,NaN,no,NaN,1,4
99313,4.071340e+09,52,yes,NaN,yes,NaN,1,4


In [6]:
C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].fillna(value=str('no'))
C_relevant['currently_enrolled'].isna().value_counts()

False    99315
Name: currently_enrolled, dtype: int64

In [7]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0506_1a_roster.dta"
)
B.sample(5)

,hhcode,idc,s1aq02,s1aq03,s1aq04,age,s1aq05a,s1aq05b,s1aq05c,s1aq06,s1aq07,s1aq08,s1aq09,s1aq10,region,province,psu
106779,4.052040e+09,58,son/daughter,male,1,3,0.0,0.0,2002.0,unmarried/never married,NaN,1,54,yes,rural,balochistan,40520403
95525,4.012010e+09,52,spouse,female,1,33,0.0,0.0,1972.0,currently married,1.0,not in roster,not alive,yes,rural,balochistan,40120104
81915,3.081210e+09,3,son/daughter,male,1,20,14.0,1.0,1985.0,unmarried/never married,NaN,1,51,yes,urban,nwfp,30812103
38841,1.302040e+09,1,head,male,1,77,0.0,0.0,1928.0,currently married,51.0,not alive,not alive,yes,rural,punjab,13020401
19961,1.121210e+09,4,son/daughter,male,1,15,0.0,1.0,1990.0,unmarried/never married,NaN,1,51,yes,urban,punjab,11212101


In [8]:
# Change the names of the questions for readability
B['sex'] = B['s1aq03']
B['marital_status'] = B['s1aq06']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [9]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [10]:
# MERGE
years05_06 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years05_06._merge.value_counts() #(ALL GOOD)

In [11]:
# Select people in school enrollment age (I would say 4-10 years old)
years05_06 = years05_06.loc[(years05_06["age"] >= 4)&(years05_06["age"] <= 10)]

In [12]:
years05_06.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    8405
2    5883
3    5274
4    3912
Name: province, dtype: int64

In [13]:
# Lets see which hh codes we have:
years05_06["hhcode"] = years05_06["hhcode"].astype("str")
years05_06['subprovince code']= years05_06['hhcode'].apply(lambda x: x[0:4])
years05_06["subprovince code"] = years05_06["subprovince code"].astype("int")
years05_06.loc[(years05_06["region"] == 1), 'region'] = 'urban'
years05_06.loc[(years05_06["region"] == 2), 'region'] = 'rural'

In [14]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0506_v1.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
22,Punjab,1232,Vehari,NaN,NaN,NaN,NaN
62,NWFP,3122,Karak,NaN,NaN,NaN,NaN
111,Sindh,2071,Sukkar,NaN,NaN,NaN,NaN
35,Sindh,2012,Khairpur,NaN,NaN,NaN,NaN
127,NWFP,3021,Peshawar,NaN,NaN,NaN,NaN


In [15]:
# MERGE
years05_06_ = pd.merge(years05_06, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years05_06_._merge.value_counts()

both          23474
left_only         0
right_only        0
Name: _merge, dtype: int64

In [16]:
years05_06_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,_merge
4513,1142030216.0,55,6,female,unmarried/never married,yes,NaN,yes,NaN,rural,1,1142,Punjab,Gujrat,NaN,NaN,NaN,NaN,both
13234,2112040114.0,53,7,male,unmarried/never married,yes,NaN,yes,NaN,rural,2,2112,Sindh,Badin,NaN,NaN,NaN,NaN,both
4406,1141340103.0,53,6,male,unmarried/never married,yes,NaN,yes,NaN,urban,1,1141,Punjab,Sargodha,NaN,NaN,NaN,NaN,both


In [17]:
# add year
years05_06_['year']=2005
years05_06_['marital_status']=years05_06_['marital_status'].astype('str')

#fix marital status categories
years05_06_.loc[(years05_06_['marital_status'] == "unmarried/never married"), 'marital_status'] = 1
years05_06_.loc[(years05_06_['marital_status'] == "currently married"), 'marital_status'] = 2
years05_06_.loc[(years05_06_['marital_status'] == "nikah solmanised but rukhsati not taken place"), 'marital_status'] = 5
years05_06_.loc[(years05_06_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years05_06_.loc[(years05_06_['marital_status'] == " divorced"), 'marital_status'] = 4
years05_06_['marital_status'].value_counts()
years05_06_['marital_status']=years05_06_['marital_status'].astype('int')

# fix column names
years05_06_['province']=years05_06_['province_y']

In [18]:
#select relevant columns
years05_06_ = years05_06_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "subprovince",
        "year",
    ]
].copy()

In [19]:
years05_06_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years05_06_.csv', index_label="id")

In [20]:
years05_06_["region"].value_counts()

rural    15332
urban     8142
Name: region, dtype: int64

In [21]:
years05_06_["currently_enrolled"] = years05_06_["currently_enrolled"].replace("yes", 1)
years05_06_["currently_enrolled"] = years05_06_["currently_enrolled"].replace("no", 0)
years05_06_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince,year
10610,2051230207.0,54,5,1,male,yes,1,urban,Sindh,2051,Karachi,2005
15551,3042010115.0,57,10,1,female,yes,1,rural,NWFP,3042,Chitral,2005
16284,3071020204.0,55,6,1,male,yes,1,urban,NWFP,3071,Mardan,2005
5568,1202020107.0,53,5,1,male,no,0,rural,Punjab,1202,Kasur,2005
7830,1332040101.0,3,10,1,male,yes,1,rural,Punjab,1332,Bahawalpur,2005


# PART 2

In [22]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer

# check region for anomalies

years05_06_["currently_enrolled"]= years05_06_["currently_enrolled"].astype('int').copy()
df_grp_1 = years05_06_.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()
df_grp_2 = years05_06_.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()

# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          328
left_only       0
right_only      0
Name: _merge, dtype: int64

In [23]:
df_grp_merge.isna().any()

sex                     False
subprovince             False
region                  False
currently_enrolled_x    False
currently_enrolled_y    False
_merge                  False
dtype: bool

In [24]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]

# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})

# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
284,female,Quetta,rural,234,77,0.329060
195,female,Faisalabad,urban,189,161,0.851852
67,male,Khanewal,urban,0,0,NaN
6,male,Bahawalnager,rural,85,53,0.623529
45,male,Hyderabad,urban,241,144,0.597510


In [25]:
df_grp_merge.to_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years05_06_merge_diff.csv")